In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from keras.preprocessing.image import load_img,img_to_array
from sklearn.metrics import mean_squared_error
from keras.initializers import RandomNormal
#from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,ReduceLROnPlateau
from keras.models import Model,Sequential
from keras.layers import *
from keras import backend as K
from keras.models import model_from_json
from matplotlib import cm as CM
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import scipy.io as io
from PIL import Image
import PIL
import h5py as h5
import os
import glob
import cv2
import numpy as np
import random
import math
import sys
from pathlib import Path


In [ ]:
root = '/u/s/s/ssrinivasan/private/CS639Project/CS639-Final-Project/'
train = os.path.join(root,'train','sequences')
test = os.path.join(root,'test','sequences')
path_sets = [train, test]

In [ ]:
path_sets

In [ ]:
img_paths_test = []
for root, dirs, files in os.walk(path_sets[1]):
        for img in files:
                img_paths_test.append(os.path.join(root, img))
test_sum = len(img_paths_test)

# img_paths_test = []
# for img_path in os.path.join(path_sets[1], '*.jpg'):
#         img_paths_test.append(str(img_path))
# print("Test image importation .. Total images : ",len(img_paths_test))
# test_sum = len(img_paths_test)

In [ ]:
img_paths_train = []
for root, dirs, files in os.walk(path_sets[0]):
        for img in files:
                img_paths_train.append(os.path.join(root, img))
train_sum = len(img_paths_train)

In [ ]:
def create_img(path):
    #Function to load,normalize and return image 
    im = Image.open(path).convert('RGB')
    """width, height = im.size 
    newsize = (width // 2, height //2)
    im = im.resize(newsize)"""
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225

    #print(im.shape)
    #im = np.expand_dims(im,axis  = 0)
    return im

def get_input(path):
    #print(path)
    path = path[0] 
    img = create_img(path)
    return(img)

def get_output(path):
    #import target
    #resize target
    #print(path)
    gt_file = h5.File(path,'r')
    target = np.asarray(gt_file['density'])
    img = cv2.resize(target,(int(target.shape[1]//8),int(target.shape[0]//8)),interpolation = cv2.INTER_CUBIC)*64
    img = np.expand_dims(img,axis  = 2)
    #print(img.shape)
    return img

def preprocess_input(image,target):
    #crop image
    #crop target
    #resize target
    crop_size = (int(image.shape[0]/2),int(image.shape[1]/2))
    #imm = np.resize(image,((image.shape[0]//2),int(image.shape[1]//2)))
    #tar = np.resize(target,((image.shape[0]//2),int(image.shape[1]//2)))
    if random.randint(0,9)<= -1:            
            dx = int(random.randint(0,1)*image.shape[0]*1./2)
            dy = int(random.randint(0,1)*image.shape[1]*1./2)
    else:
            dx = int(random.random()*image.shape[0]*1./2)
            dy = int(random.random()*image.shape[1]*1./2)

    #print(crop_size , dx , dy)
    img = image[dx : crop_size[0]+dx , dy:crop_size[1]+dy]
    
    target_aug = target[dx:crop_size[0]+dx,dy:crop_size[1]+dy]
    #print(img.shape)

    return(img,target_aug)

In [ ]:
#Image data generator 
def image_generator(files, batch_size=64):
    while True:        
        input_path = np.random.choice(a = files, size = batch_size)
        #print(input_path)
        batch_input = []
        batch_output = [] 
          
        #for input_path in batch_paths:
        
        inputt = get_input(input_path)
        output = get_output(input_path[0].replace('.jpg','.h5').replace('sequences','ground_truth') )
       
        batch_input += [inputt]
        batch_output += [output]

        batch_x = np.append(batch_x, batch_input)
        batch_y = np.append(batch_y, batch_output)
        yield (batch_x, batch_y) 

In [ ]:
def save_mod(model , str1 , str2):
    model.save_weights(str1)
    model_json = model.to_json()
    with open(str2, "w") as json_file:
        json_file.write(model_json)

In [ ]:
def init_weights_vgg(model):
    #vgg =  VGG16(weights='imagenet', include_top=False)
    
    json_file = open('/u/s/s/ssrinivasan/private/CS639Project/CS639-Final-Project/CSRNet-keras/models/V66_16.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("/u/s/s/ssrinivasan/private/CS639Project/CS639-Final-Project/CSRNet-keras/weights/VGG_16.h5")
    
    vgg = loaded_model
    
    vgg_weights=[]                         
    for layer in vgg.layers:
        if('conv' in layer.name):
            vgg_weights.append(layer.get_weights())
    offset=0
    i=0
    while(i<10): 
        if('conv' in model.layers[i+offset].name):
            model.layers[i+offset].set_weights(vgg_weights[i])
            i=i+1
            
        else:
            offset=offset+1

    return (model)

In [ ]:
def euclidean_distance_loss(y_true, y_pred):
    # Euclidean distance as a measure of loss (Loss function) 
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [ ]:
def mae(y_true, y_pred):
    return abs(K.sum(y_true) - K.sum(y_pred))
def mse(y_true, y_pred):
    return tf.math.sqrt(K.sum(y_true) * K.sum(y_pred))


In [ ]:
K.clear_session()
def CrowdNet():  
            #Variable Input Size
            rows = None
            cols = None
            
            #Batch Normalisation option
            
            batch_norm = 0
            kernel = (3, 3)
            init = RandomNormal(stddev=0.01)
            model = Sequential() 
            #custom VGG:
            
            if(batch_norm):
                model.add(Conv2D(64, kernel_size = kernel,  input_shape = (rows,cols,3),activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))#64
                model.add(BatchNormalization())
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))#128
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                model.add(BatchNormalization())
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same'))
                
            else:
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same',input_shape = (rows, cols, 3), kernel_initializer = init))
                model.add(Conv2D(64, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(128,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(256,kernel_size = kernel, activation = 'relu', padding='same', kernel_initializer = init))
                model.add(MaxPooling2D(strides=2))            
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
                model.add(Conv2D(512, kernel_size = kernel,activation = 'relu', padding='same', kernel_initializer = init))
            #Conv2D
            model.add(Dropout(0.3))
            model.add(Conv2D(512, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = "same"))
            model.add(Dropout(0.3))
            model.add(Conv2D(256, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Dropout(0.1))
            model.add(Conv2D(128, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Dropout(0.1))
            model.add(Conv2D(64, (3, 3), activation='relu', dilation_rate = 2, kernel_initializer = init, padding = 'same'))
            model.add(Conv2D(1, (1, 1), activation='relu', dilation_rate = 1, kernel_initializer = init, padding = 'same'))
        
            sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
            #model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])
            #tf.keras.losses.Huber()
            model.compile(optimizer=Adam(1e-5), loss='accuracy', metrics=[mae, "mse"])
            #optimizer=Adam(1e-5)
            model = init_weights_vgg(model)
            
            return model

In [ ]:
model = CrowdNet()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
log_dir = './log/'

In [ ]:
model.summary()

In [ ]:
checkpoint = ModelCheckpoint(log_dir + "best_weights.h5", verbose=1, monitor='val_mae',save_freq='epoch', save_weights_only=True, save_best_only=True, mode='auto')

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='mse', factor=0.2, patience=4, verbose=1)

In [ ]:
from keras.callbacks import EarlyStopping
stopit = EarlyStopping(monitor="val_mae",patience=30)

In [ ]:
callbacks = [checkpoint, reduce_lr, stopit]

In [ ]:
batch = 1 #28

In [ ]:
train_gen = image_generator(img_paths_train,batch)
train_gen

In [ ]:
val_gen = image_generator(img_paths_test,batch)

In [ ]:
sgd = SGD(lr = 1e-7, decay = (5*1e-4), momentum = 0.95)
#model.compile(optimizer=sgd, loss=euclidean_distance_loss, metrics=['mse'])
#tf.keras.losses.CosineSimilarity(axis=1) =loss
model.compile(optimizer=Adam(1e-5), loss='mae', metrics=[mae, "mse", 'accuracy'])

In [ ]:
NUM_EPOCHS = 50

In [ ]:

history = model.fit(train_gen,
    steps_per_epoch=train_sum // batch,
    validation_data=val_gen,
    validation_steps=test_sum // batch,
    epochs=NUM_EPOCHS,
    verbose = 1,
    callbacks=callbacks
    )



In [ ]:
N = NUM_EPOCHS
H=history
plt.style.use("classic")
plt.style.use("ggplot")
plt.figure(figsize = (12,4))
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")

plt.title("Training Loss on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.xticks(np.arange(0,N+2,5))

plt.legend(loc="lower left")
plt.savefig("plotLoss.png")

In [ ]:
N = NUM_EPOCHS
H=history
plt.style.use("ggplot")
plt.figure(figsize = (12,4))

plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")

plt.title("Training Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.xticks(np.arange(0,N+2,5))
#plt.yticks(np.arange(0,1,0.1))
axes = plt.gca()
#axes.xaxis.set_ticklabels(np.arange(0, N), rotation = 0, color = 'black', fontsize = 12,  verticalalignment = 'bottom')
plt.legend(loc="lower left")
plt.savefig("plotAccuracy .png")

In [ ]:
N = NUM_EPOCHS
H=history
plt.style.use("ggplot")
plt.figure(figsize = (12,4))
plt.plot(np.arange(0, N), H.history["mae"], label="train_mae")
plt.plot(np.arange(0, N), H.history["val_mae"], label="val_mae")
plt.plot(np.arange(0, N), H.history["mse"], label="train_mse")
plt.plot(np.arange(0, N), H.history["val_mse"], label="val_mse")
plt.title("Training MAE and MSE on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("MAE/MSE")
plt.xticks(np.arange(0,N+2,5))
plt.legend(loc="lower left")
plt.savefig("plotMAEMSE.png")

In [ ]:
from keras.models import model_from_json
import os
import cv2
import glob
import h5py
import pandas as pd
from sklearn.metrics import mean_absolute_error
import scipy.io as io
from PIL import Image
import numpy as np
import time

In [ ]:
N = len(img_pathss)
plt.style.use("ggplot")
plt.figure(figsize = (12,4))
plt.plot(y_true, 'b+', label = "Ground truth")
plt.plot(y_pred, 'r>', label = "Predection")


plt.title('Prediction vs GT')
plt.xlabel("Test frames")
plt.ylabel("Number of persons")
plt.xticks(np.arange(0,N+20,20))

plt.legend(loc="upper right")
plt.savefig("plotPredectionGT.png")

In [ ]:
N = len(img_pathss)
plt.style.use("ggplot")
plt.figure(figsize = (12,4))

error = np.array(y_pred) - np.array(y_true)
plt.plot(error, label = "Counting error")

plt.title('Pred - GT, mean = {}, MAE={}'.format(
    str(round(np.mean(error), 3)),
    str(round(np.mean(np.abs(error)), 3))
))
plt.xlabel("Test frames")
plt.ylabel("Predection error")
plt.xticks(np.arange(0,N+20,20))

plt.legend(loc="lower left")
plt.savefig("plotLoss.png")

In [ ]:
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))
print("MAE : " , ans )